In [2]:
import cv2
import numpy as np
import json
from collections import defaultdict

In [4]:
# 1-drop IDs with fewer than X occurrences

FPS = 25
video_file='footballTracking\Barcelona Tiki-Taka vs Real Madrid _Barca are back!.mp4'
output_folder='output'
min_occurrences=30
eps=15 # For DBSCAN clustering
min_samples=3 
heatmap_image = cv2.imread("footballTracking\map_heat.png")


""" Load the player data from the JSON file """
file_name= "footballTracking/barca_fixes.json"
with open(file_name, "r") as json_file:
    data = json.load(json_file)

def transform_data_to_lists(frame_data_dict):
    frames = sorted(frame_data_dict.keys(), key=lambda x: int(x))
    all_projected_points = []
    all_MapIDs = []
    
    for frame in frames:
        frame_data = frame_data_dict[frame]
        projected_points = []
        MapIDs = []
        
        for ID, coords in frame_data.items():
            MapIDs.append(int(ID))
            projected_points.append((coords["x"], coords["y"]))
        
        all_projected_points.append(projected_points)
        all_MapIDs.append(MapIDs)
    
    return all_projected_points, all_MapIDs 
    
def drop_unreliable_ids(all_projected_points , all_MapIDs , data):
    
    """ Remove player IDs with fewer than `min_occurrences` occurrences """
    occurrences = defaultdict(int)
    
    # Count the number of occurrences for each player ID
    for frame_num in all_MapIDs:
        for player_id in frame_num:
            temp_id = occurrences[player_id]
            temp_id+=1
            occurrences[player_id] =temp_id

    
    #print(occurrences ,"\n",len(occurrences),"\n",type(occurrences))
    #print(occurrences[4])
    
    filtered_data =defaultdict(int)
    list_id=[]
    list_coords=[]
    frames = sorted(data.keys(), key=lambda x: int(x))
    for frame in frames:
        filtered_data =defaultdict(int)
        frame_data = data[frame]
        #print(data[frame])
        for ID, coords in frame_data.items():
            if occurrences[int(ID)] >= min_occurrences:
                list_id.append(ID) 
                list_coords.append(coords)
                #print(filtered_data.keys(),filtered_data.values())
                #print(filtered_data[ID],"frame update" , coords)
            else:
                continue
                
        #list.append(filtered_data) 
    #print(list_id ,"\n\n\n\n",len(list_id))
    #print(list_coords ,"\n\n\n\n",len(list_coords))
    return list_id ,list_coords


all_projected_points, all_MapIDs  = transform_data_to_lists(data)
filtered_id, filtered_coords = drop_unreliable_ids(all_projected_points, all_MapIDs ,data)

In [6]:
 print(filtered_id,filtered_coords)

['4', '5', '1', '7', '3', '8', '2', '6', '11', '9', '17', '10', '18', '5', '1', '3', '4', '18', '2', '8', '6', '7', '9', '10', '17', '19', '21', '11', '22', '5', '3', '1', '19', '7', '4', '8', '2', '6', '9', '10', '21', '11', '18', '22', '17', '4', '5', '3', '1', '18', '19', '6', '10', '7', '22', '17', '11', '2', '21', '8', '9', '5', '4', '1', '6', '18', '10', '7', '8', '22', '21', '17', '2', '19', '3', '9', '11', '5', '4', '18', '6', '10', '1', '19', '7', '8', '2', '22', '21', '11', '17', '3', '9', '5', '19', '4', '6', '1', '7', '18', '21', '10', '22', '11', '2', '9', '8', '28', '17', '3', '27', '5', '19', '7', '1', '17', '10', '22', '6', '4', '28', '3', '11', '21', '2', '8', '29', '9', '18', '27', '5', '1', '6', '3', '7', '28', '8', '10', '4', '17', '19', '22', '2', '9', '11', '29', '18', '21', '27', '1', '5', '28', '3', '2', '7', '19', '6', '22', '4', '9', '21', '11', '8', '29', '17', '18', '10', '27', '1', '7', '19', '2', '5', '3', '9', '6', '28', '11', '22', '17', '4', '10', '21',

In [8]:
#transform list of ids from str to int 
for i in range(len(filtered_id)):
    filtered_id[i]= int(filtered_id[i])

print (filtered_id)

[4, 5, 1, 7, 3, 8, 2, 6, 11, 9, 17, 10, 18, 5, 1, 3, 4, 18, 2, 8, 6, 7, 9, 10, 17, 19, 21, 11, 22, 5, 3, 1, 19, 7, 4, 8, 2, 6, 9, 10, 21, 11, 18, 22, 17, 4, 5, 3, 1, 18, 19, 6, 10, 7, 22, 17, 11, 2, 21, 8, 9, 5, 4, 1, 6, 18, 10, 7, 8, 22, 21, 17, 2, 19, 3, 9, 11, 5, 4, 18, 6, 10, 1, 19, 7, 8, 2, 22, 21, 11, 17, 3, 9, 5, 19, 4, 6, 1, 7, 18, 21, 10, 22, 11, 2, 9, 8, 28, 17, 3, 27, 5, 19, 7, 1, 17, 10, 22, 6, 4, 28, 3, 11, 21, 2, 8, 29, 9, 18, 27, 5, 1, 6, 3, 7, 28, 8, 10, 4, 17, 19, 22, 2, 9, 11, 29, 18, 21, 27, 1, 5, 28, 3, 2, 7, 19, 6, 22, 4, 9, 21, 11, 8, 29, 17, 18, 10, 27, 1, 7, 19, 2, 5, 3, 9, 6, 28, 11, 22, 17, 4, 10, 21, 29, 18, 8, 27, 19, 7, 17, 4, 2, 1, 22, 9, 3, 6, 21, 29, 5, 28, 10, 11, 27, 8, 18, 1, 4, 22, 5, 7, 17, 3, 21, 10, 19, 6, 29, 2, 9, 28, 18, 11, 27, 8, 17, 9, 4, 7, 19, 1, 2, 22, 5, 3, 11, 6, 10, 21, 28, 18, 8, 29, 27, 9, 17, 7, 19, 1, 5, 4, 3, 29, 11, 6, 10, 22, 2, 28, 8, 21, 18, 27, 9, 17, 7, 5, 6, 10, 19, 1, 22, 3, 4, 11, 8, 21, 29, 18, 2, 28, 27, 17, 9, 7, 5, 8,

In [10]:
#transfer coordinates from dict to list of lists each list contain (x,y )cooresponding

nested_list_of_xy= []
for i in filtered_coords:
    nested_list_of_xy.append([])
#print (len(nested_list_of_xy))
for i in range(len(filtered_coords)):
    item= filtered_coords[i]
    for j in range(1):
        #index =filtered_coords.index(i)
        #print(i)
        nested_list_of_xy[i].append(item['x'])
        nested_list_of_xy[i].append(item['y'])
print(len(nested_list_of_xy))
print(nested_list_of_xy[:30])

16681
[[214, 262], [231, 223], [267, 231], [245, 147], [254, 189], [240, 199], [234, 135], [218, 224], [280, 92], [178, 242], [265, 134], [194, 193], [182, 157], [228, 222], [251, 219], [244, 186], [215, 267], [181, 155], [232, 137], [235, 196], [218, 227], [239, 149], [174, 269], [194, 200], [250, 139], [209, 213], [175, 232], [258, 110], [207, 186], [231, 223]]
